# HW 2: Data Preparation for Sentiment Classification

In this homework we will prepare the IMDB movie review sentiment dataset. We will prepare it to fit a model that will predict wether a new reviews has positive or negative sentiment. These sentences are movie reviews and the label reflects whether it is a 'positive' review or a 'negative' review. **Start by downloading the IMDB_Dataset into a pandas DataFrame**

In [ ]:
import pandas as pd
import numpy as np

#Download the dataset into a Pandas DataFrame and display the first 5 rows
## YOUR CODE HERE
imdb = 

We have to process the data first, so that we have fixed length sequences.

<b>We want to split the dataset into classes and labels. </b>

In [ ]:
imdb_x = 
imdb_y = 

In [ ]:
# Paste the toBinary functon created in HW 1
def toBinary(data, positive):
    ## YOUR CODE HERE

**Use the toBinary method to tranform the sentiment column into binary, 1 for positive and 0 for negative.**

In [ ]:
toBinary(imdb_y, 'positive')
imdb.head()

"Lemmatization is the process of converting a word to its base form. The difference between stemming and lemmatization is, lemmatization considers the context and converts the word to its meaningful base form, whereas stemming just removes the last few characters, often leading to incorrect meanings and spelling errors."
    https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

Lemmatize the sentences using any library from the article. Make sure to filter out non-alphabetical characters. 

In [ ]:
#import

def Lemmatize(data):
    # Init the Wordnet Lemmatizer
    data = data.str.split(" |,|<br /><br />|\"|\'|!") # splits the data from sentences to words, feel free to change
    lemmatizer = # Initialize lemmatizer

    # YOUR CODE HERE

    return data

Lemmatize(imdb_x)

The data has to be put into integer form, so each integer represents a unique word, 0 represents a PAD character, 1 represents a START character and 2 represents a character that is unknown because it is not in the top `num_words`. 
Thus 3 represents the first real word. 

Also the words should be in decreasing order of frequency, so the word that 3 represents is the most common word in the dataset. 

Complete CreateDict which will take in a data column <br>
    1) Create a Dict that maps {Word: Apperances in dataset} <br>
    2) Choose the top N most recurring words and give ascending indexes starting at 3 <br>

In [ ]:
numWords = 1000

def CreateDict(data, topN):

    # YOUR CODE HERE
        
    return dict(wordCounter)
    
wordCounter = CreateDict(imdb_x, numWords)

Complete  replaceByIndex which will replace known words with their index and unknown words with a 2.

In [ ]:
def replaceByIndex(data, wordCounter):
    
    # YOUR CODE HERE
    
    pass 

imdb_x = replaceByIndex(imdb_x, wordCounter)

In [ ]:
imdb_x.head()

#### We want to process the data into NumPy arrays of sequences that are all length 200. If a given sequence is shorter than 200 tokens we want to pad the rest of the sequence out with zeros so that the sequence is 200 long. If the sequence is longer than 200 we want to cut it down to length 200. We also want to add a 1 at the beggining of every sentence (after 0s) to signal the beggining. 

In [ ]:
def process_data(data):
    
    #YOUR CODE HERE 
    
    return processed

imdb_x = process_data(imdb_x)

** separate the dataset into train and test sets, test set should be 1/3 of the set. ** <p>
This sklearn method will make your life much easier: 
[train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
## YOUR CODE HERE

x_train_proc, x_test_proc, y_train, y_test = #YOUR CODE HERE

At this point **your job is done!!!** Congratulations, if done correctly, the sentences are processed and ready to be used as features and labels to train a Recurrent Neural Network (LSTM). You will learn how to do this yourself in the next couple weeks. For now, you can just sit back and "follow along" as we build this model using Keras and then train it. 

The first thing we will do is initialize the model using Sequential.

In [ ]:
import keras
from keras import Sequential

imdb_model = Sequential()

Now we want to add an embedding layer. The purpose of an embedding layer is to take a sequence of integers representing words in our case and turn each integer into a dense vector in some embedding space. (This is essentially the idea of Word2Vec). We want to create an embedding layer with vocab size equal to the max num words we allowed when we loaded the data (in this case 1000), and a fixed dense vector of size 32. Then we have to specify the max length of our sequences and we want to mask out zeros in our sequence since we used zero to pad.
Use the docs for embedding layer to fill out the missing entries: https://keras.io/layers/embeddings/

In [ ]:
from keras.layers.embeddings import Embedding
imdb_model.add(Embedding(2000, 32, input_length=200, mask_zero=True))

#### **(a)** We add an LSTM layer with 32 outputs, then a Dense layer with 16 neurons, then a relu activation, then a dense layer with 1 neuron, then a sigmoid activation. Then we print out the model summary. The Keras documentation is here: https://keras.io/

In [ ]:
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Activation
imdb_model.add(LSTM(32))

In [ ]:
imdb_model.add(Dense(units=16, activation='relu'))
imdb_model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
imdb_model.summary()

#### **(b)** Now we compile the model with binary cross entropy, and the adam optimizer. We include accuracy as a metric in the compile. Then train the model on the processed data.

In [ ]:
imdb_model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['acc'])

In [ ]:
imdb_model.fit(x_train_proc, y_train)

In [ ]:
print("Accuracy: ", imdb_model.evaluate(x_test_proc, y_test)[1])

## If you did the data pre-processing correctly you should be getting around an 80% accuracy. congratulations, that is much better than random! If you are getting a test accuracy that is significantly lower, you probably did something wrong. 

#### Now we can look at our predictions and the sentences they correspond to.

In [ ]:
y_pred = imdb_model.predict(x_test_proc)

In [ ]:
word_to_id = wordCounter
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

id_to_word = {value:key for key,value in word_to_id.items() if value < 2000}
def get_words(token_sequence):
    return ' '.join(id_to_word[token] for token in token_sequence)

def get_sentiment(y_pred, index):
    return 'Positive' if y_pred[index] else 'Negative'

In [ ]:
y_test = [i for i in y_test]
y_pred = np.vectorize(lambda x: int(x >= 0.5))(y_pred)
correct = []
incorrect = []
for i, pred in enumerate(y_pred):
    if y_test[i] == pred:
        correct.append(i)
    else:
        incorrect.append(i)

#### Now we print out one of the sequences we got correct.

In [ ]:
print(get_sentiment(y_pred, correct[10]))
print(get_words(x_test_proc[correct[10]]))

#### And one we got wrong.

In [ ]:
print(get_sentiment(y_pred, incorrect[10]))
print(get_words(x_test_proc[incorrect[10]]))

#### As you can see the amount of UNKNOWN characters in the sequence cause by having only 1000 vocab words is hurting our performance. If you want, go back and increase the number of vocab words to 2000 and compare your accuracy. (Optional)

## Embedding Exploration
#### Another interesting thing to do is see if our learned embeddings mean anything reasonable.

In [ ]:
from keras import backend as K

In [ ]:
# this function takes a list of token sequences as inputs 
# and outputs the corresponding vector outputs of our `Embedding` layer
embedding_func = K.function([imdb_model.inputs[0]], [imdb_model.layers[0].output])

In [ ]:
# this function outputs the embedding of a given word using above function
def word_to_embedding(word):
    token = word_to_id[word]
    seq = [token]
    sequences = [seq]
    inputs = [process_data(sequences)]
    embedding = embedding_func(inputs)
    return embedding[0][0][0]

In [ ]:
valid_words = [word for word, token in word_to_id.items() if token < 1000]

In [ ]:
valid_word_embeddings = {word: word_to_embedding(word) for word in valid_words}

Since we used an embedding layer with an output size of 32, our embeddings are going to be 32-dimensional vectors. Humans can't effectively visualize beyond 3 (maybe 4) dimensions so we want to use a dimensionality reduction technique to make our embeddings more visualizable. One such technique is Principal Component Analysis or PCA. The library scikit-learn provides an easy to use API for this technique.

In [ ]:
import sklearn
from sklearn import decomposition

#### using the documentation for scikit-learn's PCA [here](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html): create a PCA object with `n_components=2`

In [ ]:
pca = decomposition.PCA(n_components=2)

#### using the same documentation find the function to fit the PCA transform to the provided embedding vectors. This step essentially. For the curious, this step essentially finds the 2 dimensions (since we specified `n_components=2` that explain the most variance of the dataset, in other words the two dimensions that are most representative of the deviations of any one sample to another. So these 2 dimensions are the most important and therefore the best to visualize.

In [ ]:
vectors_to_fit = valid_word_embeddings.values()
pca.fit(list(vectors_to_fit))

#### Now we want to visualize our embeddings in these new PCA dimensions, so using the same documentation from above fill out the missing spots in the code below to transform the embeddings into the pca dimensions.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def get_pca_words(words):
    embeddings = [valid_word_embeddings[word] for word in words]
    pcas = [pca.transform(embedding.reshape(1, -1)) for embedding in embeddings]
    return pcas

def plot_pca_words(words, scale=1):
    pcas = get_pca_words(words)
    zeros = [0 for _ in pcas]
    x_start = zeros
    y_start = zeros
    xs = [p[0, 0] for p in pcas]
    ys = [p[0, 1] for p in pcas]
    plt.quiver(x_start, y_start, xs, ys, scale=scale)
    plt.show()

#### Now using the above functions we can plot the corresponding pca vectors of any words we like. Below are some good examples of pairs of words that are similar within the movie review context and their corresponding vectors are also similar. **This is a good sign.** This means the embedding we have learned is likely doing something somewhat reasonable.

In [ ]:
plot_pca_words(['film', 'entertainment'], scale=0.5)

In [ ]:
plot_pca_words(['man', 'woman'])

In [ ]:
plot_pca_words(['good', 'bad', 'horrible', 'great'], scale=2)

#### Now find 2 more pairs of words that are similar in PCA'd embedding space. 

In [ ]:
plot_pca_words(['brilliant', 'good'], scale=2)

In [ ]:
plot_pca_words(['annoying', 'worst'], scale=2)

#### Given that the task we learned these embeddings for was sentiment classification, the embeddings are typically more meaningful for adjectives. Write a sentence or two about why you think this last statement makes sense intuitively.

#TODO see above

> It makes sense that the most meaningful embeddings are the adjectives because those are what the network should 
be looking out for to analyze sentiment. It is analyzing moview reviews and therefore the most meaningful and expressive words are going to be the adjectives.

#### Now just for fun we can write a function that gives us the 10 closest words to a provided word.

In [ ]:
def word_to_angle(word):
    p = pca.transform(valid_word_embeddings[word].reshape(-1, 1))
    return np.arctan(p[0, 1] / p[0, 0])
valid_word_angles = [word_to_angle(word) for word in valid_words]

In [ ]:
def find_closest_n(value, n):
    indices = np.argsort(np.abs(np.array(valid_word_angles) - value))
    return [(valid_words[ind], valid_word_angles[ind]) for ind in indices[:n]]

In [ ]:
find_closest_n(word_to_angle('terrible'), 10) # TRY IT YOURSELF